In [ ]:
#google driveとの連携
from google.colab import drive
drive.mount('/content/drive')

#カレントディレクトリの設定
dir_path = '/content/drive/My Drive/Colab Notebooks/repo10/'
import os
os.chdir(dir_path)

#モジュール追加用pathの設定
import sys
sys.path.append('..')

#モジュールのインポート
import numpy as np

**80.py**

In [ ]:
import re
from collections import Counter
import pickle

# 単語の最小出現回数
min_freq = 2

train_path = './data/train.txt'#todo 学習用ファイルパス
word2id_path = './data/train.pt' #todo 出力先ファイルパス


# タイトルに含まれる単語をすべてリストword_listに格納
word_list = []
with open(train_path, encoding="utf-8") as f:
    for line in f:
        if not line:
            continue
        title = line.split('\t')[1]#todo タイトルを取得
        for word in title.split():
            word_list.append(word)

word_count = Counter(word_list)#todo Counterを使ってword_listの各単語の出現回数をカウント
# print('word_count', word_count) #word_count Counter({'to': 2098, '...': 1991, 'in': 1424, ...
# 単語をIDに変換するためのdict．keyを単語，valueをIDにする．
word2id = {}

id = 1
for word, freq in word_count.items():
    if freq < min_freq: #todo その単語の出現頻度がmin_freqよりも小さい場合:
        word2id[word] = 0#todo 単語のIDは0とする
    else:
        word2id[word] = id #todo 単語にIDを割り当てる
        id += 1#todo IDをインクリメント

        
# 表示
print("word_num", len(word2id))
# 保存
pickle.dump(word2id, open(word2id_path, "wb"))


word_num 23620


**83.py**

In [ ]:
print(word2id)

{'UPDATE': 1, '1-BioDelivery,': 0, 'Endo': 3, 'painkiller': 4, 'effective': 5, 'in': 6, 'study': 7, 'Southern': 8, 'and': 9, 'Turner': 0, 'to': 11, 'Buy': 12, 'Macho': 0, 'Springs': 0, 'From': 15, 'First': 16, 'Solar': 17, 'Twitter': 18, 'forced': 19, 'shut': 20, 'down': 21, 'Tweetdeck': 0, 'app': 23, 'amid': 24, 'major': 25, 'security': 26, 'alert': 27, 'JD.com': 28, 'Goes': 29, 'Public,': 0, 'China': 31, 'Has': 32, 'a': 33, 'New': 34, 'Internet': 35, 'Billionaire': 36, 'Amazon': 37, 'Looks': 38, 'Like': 39, "It's": 40, 'Creating': 41, 'A': 42, 'Netflix': 43, 'For': 44, 'Books': 45, 'Cancer': 46, 'Type': 47, 'May': 48, 'Be': 49, 'Linked': 50, 'With': 51, 'Socioeconomic': 0, 'Status': 53, 'Leaked': 54, 'iPhone': 55, '6': 56, 'Photos': 57, 'Appear': 58, 'To': 59, 'Show': 60, 'Design': 61, 'In': 62, 'All': 63, 'Its': 64, 'Thin,': 0, 'Rounded': 0, '...': 67, 'Hess': 68, 'deal,': 69, 'Marathon': 70, 'fuel': 71, 'traders': 72, 'gain': 73, 'big': 74, 'E.Coast': 0, 'foothold': 76, 'Mel': 77, 

In [ ]:
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence

In [ ]:
word2id_path = './data/train.pt' #todo 80.pyで保存したファイルパスを指定
word2id = pickle.load(open(word2id_path, "rb"))

# word_listの単語リストをIDリストに変換する関数
def get_ids(word_list):
    ids = []
    for word in word_list:
        if word in word2id: #todo wordがword2idのキーに含まれる場合，
            ids.append(word2id[word]) #todo そのIDをidsにappend
        else:
            ids.append(0) #todo wordがword2idのキーに含まれない場合，0をidsにappend
    return ids


def get_data(fname):
    label_list = [] # ラベル(カテゴリーのid)を格納するリスト
    ids_list = [] # タイトルに含まれる単語のID系列のtensorを格納するリスト

    category2num = {"b": 0, "t": 1, "e": 2, "m": 3}

    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.strip()
            title = line.split('\t')[1]#todo lineからタイトルを取得
            word_list =  title.split() #todo titleを単語に分割
            ids = get_ids(word_list)
            ids_tensor = torch.tensor(ids)#todo idsをtensorに変換
            ids_list.append(ids_tensor)#todo ids_tensorをids_listにappend

            category = line.split('\t')[0]#todo lineからカテゴリーを取得
            label = category2num[category] #todo category2numでカテゴリーをラベルに変換
            label_list.append(label) #todo labelをlabel_listにappend
          
    labels = torch.tensor(label_list) #todo label_listをtensorに変換
    return ids_list, labels


class ClassifierRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size):
        super(ClassifierRNN, self).__init__()
        self.embedding = #todo 単語埋め込み層
        self.rnn = #todo LSTM層
        self.linear = #todo 線形層
    
    def forward(self, x, x_len):  
        embed_x = #todo xを単語埋め込み

        packed_x = #todo embed_xとx_lenを用いてPackedSequence型のデータに変換

        _, (h, _) = #todo LSTM層に入力
        output = #todo 線形層に入力

        output = #todo squeeze()を使用し，outputの形状が[1, batch_size, target_size]となっているものを[batch_size, target_size]に変換

        #print(output.size()) #確認用(確認する場合はコメントアウトを解除)
        return output


class RnnDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.length = #todo dataに含まれる各ID系列の長さのリスト
        self.label = label

        self.pad_data = #todo dataをpadding

    def __len__(self):
        return len(self.pad_data)

    def __getitem__(self, idx):
        return #todo idxに対応するpad_data, length, labelの3つを返り値とする



emb_dim = 300
hidden_dim = 50
target_size = 4
batch_size = 64

vocab_size = max(word2id.values()) + 1 #0のやつはカウントしない

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データの準備
train_path = './data/train.txt'#todo 学習用ファイルパス
train_x, train_y = get_data(train_path)
valid_path = './data/valid.txt' #todo 評価用ファイルパス
valid_x, valid_y = get_data(valid_path)


dataset = RnnDataset(train_x, train_y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = ClassifierRNN(vocab_size, emb_dim, hidden_dim, target_size)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


def train(model, train_loader, len_train):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for data, data_len, target in train_loader:
        # GPUへ
        data = data.to(device)
        data_len = data_len.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        pred = #todo modelで予測
        loss = #todo lossを計算
        loss.backward()
        optimizer.step()

        total_loss #todo lossをtotal_lossに加算
        correct_num #todo 予測の正解数をカウント
    
    batch_loss = total_loss/len_loader #バッチごとの平均ロス
    acc = #todo 正解率
    return batch_loss, acc


# validデータはそのままGPUへ
valid_x_len = #todo valid_xの各要素の長さのリスト

valid_pad_x = #todo valid_xをpadding

# GPUへ
valid_x_len = valid_x_len.to(device)
valid_pad_x = valid_pad_x.to(device)
valid_y = valid_y.to(device)



def evaluation(model, data, data_len, target):
    model.eval()
    with torch.no_grad():
        pred = #todo 予測
        loss = #todo 損失を計算
        acc = #todo 正解率を計算
    return loss.item(), acc


len_train = len(train_y)

for epoch in range(30):
    train_loss, train_acc = train(model, train_loader, len_train)
    valid_loss, valid_acc = evaluation(model, valid_pad_x, valid_x_len, valid_y)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    print(f"<valid> Loss: {valid_loss}\tAccuracy: {valid_acc}")

Reference
1. https://note.nkmk.me/python-collections-counter/
2. https://www.headboost.jp/python-counter/#2_Counter